<a href="https://colab.research.google.com/github/mianyumifen-bot/codePublic/blob/main/%E2%80%9C%E7%AC%AC%E5%9B%9B%E5%A4%A9sentialGCC%E6%94%B9%E4%B8%BAallData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安装必要包（只需运行一次）
!pip install --quiet earthengine-api geemap rasterio rioxarray geopandas shapely pyproj


In [ ]:
# 授权并挂载 Google Drive，然后初始化 Earth Engine
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, time, json, math
import geopandas as gpd
import pandas as pd
import ee
import geemap

# Earth Engine 授权并初始化（会弹出链接）
try:
    ee.Initialize(project="ee-mianyumifen")
    print("Earth Engine 已初始化。")
except Exception as e:
    print("需要 EE 授权，开始交互式授权流程...")
    ee.Authenticate()
    ee.Initialize(project="ee-mianyumifen")
    print("Earth Engine 已授权并初始化。")

# 常用打印确认
print("当前工作目录：", os.getcwd())
print("Drive 挂载路径：/content/drive")


Mounted at /content/drive
Earth Engine 已初始化。
当前工作目录： /content
Drive 挂载路径：/content/drive


In [ ]:
# ========== 请修改下面两个路径为你 Drive 中实际的位置 ==========
DRIVE_INPUT_DIR = '/content/drive/MyDrive/allRoi'   # 放 allRoi.shp 的目录
OUT_DRIVE_FOLDER = 'sential2GCC_exports_final_allData'                        # 导出到 Drive 的文件夹名（在 MyDrive 下）
LOG_PATH = os.path.join(DRIVE_INPUT_DIR, 'sential2_export_log.json')  # 导出日志路径

# 其他控制参数
HLS_COLLECTION = 'COPERNICUS/S2_SR_HARMONIZED'  # sential2 数据集

PREFERRED_BANDS = ['B4', 'B3', 'B2', 'B11'] # Red, Green, Blue, and B6 (SWIR1)

EXPORT_SCALE = 10 # meters（sential的近红外是20m的）
MAX_BANDS_WARN = 12000  # 如果堆栈波段超过此数，脚本会改为按月合成导出以避免过大单文件
SLEEP_BETWEEN_START = 1  # 启动导出任务时的间隔秒数（防并发过高）


In [ ]:
# 读取 ROI shapefile（确保 .shp/.dbf/.shx/.prj 都在同一目录）
roi_shp_path = os.path.join(DRIVE_INPUT_DIR, 'allRoi.shp')
if not os.path.exists(roi_shp_path):
    raise FileNotFoundError(f"找不到 {roi_shp_path}，请把 allRoi.shp 放到 DRIVE_INPUT_DIR 指定的目录下。")

rois_gdf = gpd.read_file(roi_shp_path)
print("读取 ROI 数量：", len(rois_gdf))
print("字段名示例：", rois_gdf.columns.tolist())

# 选择名称字段（优先 'site' 或 'name'，否则创建基于索引的 site 字段）
if 'site' in rois_gdf.columns:
    name_field = 'site'
elif 'name' in rois_gdf.columns:
    name_field = 'name'
else:
    name_field = 'site'
    rois_gdf[name_field] = [f"roi_{i}" for i in range(len(rois_gdf))]
    print("未找到 name/site 字段，已自动生成 'site' 字段。")

# 构建字典：site_name -> shapely geometry
roi_dict = {str(row[name_field]): row.geometry for _, row in rois_gdf.iterrows()}
print("示例 site keys:", list(roi_dict.keys())[:10])


读取 ROI 数量： 11
字段名示例： ['name', 'Shape_Leng', 'Shape_Le_1', 'Shape_Area', 'geometry']
示例 site keys: ['northinletsaltmarsh', 'norriepoint', 'mayberry', 'hillslough', 'gcesapelo', 'cmarshhighsaltmarsh', 'brackishimpoundment', 'bnzrichfen', 'richmondbrackishmarsh', 'siwetland']


In [ ]:
# 直接使用用户提供的时间表（字典）
# ---- 用下面这段替换旧的 schedule_dict + pairs 创建代码块 ----

# 要下载的站点列表（需要时可修改）
target_sites = [
    'northinletsaltmarsh',
    'norriepoint',
    'mayberry',
    'hillslough',
    'gcesapelo',
    'cmarshhighsaltmarsh',
    'brackishimpoundment',
    'bnzrichfen',
    'richmondbrackishmarsh',
    'siwetland',
    'vancouversaltmarsh'
]

import datetime
current_year = datetime.datetime.now().year

pairs = []   # 用来保存 (site, year) 的列表

for site in target_sites:
    if site not in roi_dict:
        print(f"  警告：在 shapefile 中未找到站点：{site}")
        continue

    ee_roi = shapely_to_ee(roi_dict[site])
    col = ee.ImageCollection(HLS_COLLECTION).filterBounds(ee_roi)

    # 检查该站点是否有任何影像
    col_size = col.size().getInfo()
    print(f"[{site}] 全部年份影像数量:", col_size)
    if col_size == 0:
        print(f"  {site} 没有找到 Sentinel-2 影像，跳过。")
        continue

    # 取按时间排序的第一张影像，获取其时间戳并转成年份
    first_img = col.sort('system:time_start').first()
    try:
        first_ts = first_img.get('system:time_start').getInfo()
        first_year = datetime.datetime.utcfromtimestamp(first_ts / 1000).year
    except Exception as e:
        print(f"  无法确定 {site} 的首年（错误：{e}），默认用 2015 年。")
        first_year = 2015

    # 从首年到当前年（包含当前年）都创建 pairs
    for y in range(first_year, current_year + 1):
        pairs.append((site, str(y)))

print("构造好的 site-year 对数量:", len(pairs))
print(pairs[:40])  # 预览前 40 项



[northinletsaltmarsh] 全部年份影像数量: 2502


/tmp/ipython-input-3265127447.py:43: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  first_year = datetime.datetime.utcfromtimestamp(first_ts / 1000).year


[norriepoint] 全部年份影像数量: 1343
[mayberry] 全部年份影像数量: 1341
[hillslough] 全部年份影像数量: 675
[gcesapelo] 全部年份影像数量: 621
[cmarshhighsaltmarsh] 全部年份影像数量: 1247
[brackishimpoundment] 全部年份影像数量: 1253
[bnzrichfen] 全部年份影像数量: 1163
[richmondbrackishmarsh] 全部年份影像数量: 1255
[siwetland] 全部年份影像数量: 1341
[vancouversaltmarsh] 全部年份影像数量: 1254
构造好的 site-year 对数量: 118
[('northinletsaltmarsh', '2015'), ('northinletsaltmarsh', '2016'), ('northinletsaltmarsh', '2017'), ('northinletsaltmarsh', '2018'), ('northinletsaltmarsh', '2019'), ('northinletsaltmarsh', '2020'), ('northinletsaltmarsh', '2021'), ('northinletsaltmarsh', '2022'), ('northinletsaltmarsh', '2023'), ('northinletsaltmarsh', '2024'), ('northinletsaltmarsh', '2025'), ('norriepoint', '2015'), ('norriepoint', '2016'), ('norriepoint', '2017'), ('norriepoint', '2018'), ('norriepoint', '2019'), ('norriepoint', '2020'), ('norriepoint', '2021'), ('norriepoint', '2022'), ('norriepoint', '2023'), ('norriepoint', '2024'), ('norriepoint', '2025'), ('mayberry', '2015'), ('m

In [ ]:
# ---------- 辅助：把 shapely geometry 转为 ee.Geometry ----------
from shapely.geometry import mapping
def shapely_to_ee(geom):
    return ee.Geometry(mapping(geom))

# ---------- 生成掩膜函数（优先用 Fmask / QA_PIXEL，否则用经验阈值） ----------
def make_mask_function(bandnames_list):
    """
    Return a mask function adapted for Sentinel-2 SR Harmonized (QA60) and generic fallbacks.
    - If QA60 present -> use bits 10 (opaque clouds) and 11 (cirrus) to mask.
    - Else if Fmask/SCL etc present, try sensible handling.
    - Otherwise fallback to simple spectral rules (MNDWI + brightness).
    """
    bandset = set(bandnames_list)
    use_qa60 = 'QA60' in bandset or 'qa60' in bandset or 'Qa60' in bandset
    use_fmask = 'Fmask' in bandset or 'fmask' in bandset
    use_scl = 'SCL' in bandset  # scene classification layer may exist

    def mask_fn(img):
        img = ee.Image(img)

        # 1) If QA60 present -> use bits 10 and 11
        if use_qa60:
            # QA60: bit 10 = opaque clouds, bit 11 = cirrus
            qa_key = 'QA60' if 'QA60' in bandset else ('qa60' if 'qa60' in bandset else 'Qa60')
            qa = img.select(qa_key)
            opaque_cloud_bit = 1 << 10
            cirrus_bit = 1 << 11
            no_opaque = qa.bitwiseAnd(opaque_cloud_bit).eq(0)
            no_cirrus = qa.bitwiseAnd(cirrus_bit).eq(0)
            qa_mask = no_opaque.And(no_cirrus)
            return img.updateMask(qa_mask)

        # 2) If Fmask present -> follow earlier Fmask logic (if user has fmask)
        if use_fmask:
            key = 'Fmask' if 'Fmask' in bandset else 'fmask'
            fmask = img.select(key)
            # adjust bit positions according to your Fmask definition (example earlier used bits 1,3,4,5)
            cloud_bit = 1 << 1
            cloud_shadow_bit = 1 << 3
            snow_bit = 1 << 4
            water_bit = 1 << 5
            is_cloud = fmask.bitwiseAnd(cloud_bit).neq(0)
            is_cloud_shadow = fmask.bitwiseAnd(cloud_shadow_bit).neq(0)
            is_snow = fmask.bitwiseAnd(snow_bit).neq(0)
            is_water = fmask.bitwiseAnd(water_bit).neq(0)
            clear_mask = is_cloud.Not().And(is_cloud_shadow.Not()).And(is_snow.Not()).And(is_water.Not())
            return img.updateMask(clear_mask)

        # 3) If SCL (scene classification) exists -> mask cloud classes (optional)
        if use_scl:
            scl = img.select('SCL')
            # Typical SCL classes to mask: 3 = cloud shadow, 8 = medium probability cloud,
            # 9 = high probability cloud, 10 = thin cirrus, 11 = snow. This may vary by dataset.
            cloud_mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
            return img.updateMask(cloud_mask)

        # 4) Fallback: spectral rules using MNDWI and brightness (conservative)
        try:
    # select, cast to float and convert to reflectance (scale factor 0.0001)
          MNDWI = img.select(['B3','B11']).toFloat().multiply(0.0001).normalizedDifference(['B3','B11']).rename('MNDWI')
        except Exception:
          MNDWI = ee.Image(0)

        # brightness using whichever of preferred bands exist
        bnames = [b for b in PREFERRED_BANDS if b in bandnames_list]
        if len(bnames) >= 3:
    # cast + scale then compute mean brightness (reflectance domain)
          brightness = img.select(bnames[:3]).toFloat().multiply(0.0001).reduce(ee.Reducer.mean()).rename('BRIGHT')
        else:
          brightness = ee.Image(0)

        bright_ok = brightness.lt(0.9)   # filter out extremely bright pixels (likely cloud)
        not_water = MNDWI.lt(-0.1)       # keep non-water (conservative)
        final_mask = bright_ok.And(not_water)
        return img.updateMask(final_mask)

    return mask_fn

# ---------- 处理单 site-year 的函数 ----------
def process_site_year(site_name, year, roi_geom, out_drive_folder=OUT_DRIVE_FOLDER, scale=EXPORT_SCALE):
    """
    Process a single site-year:
    - filter Sentinel-2 by date & roi
    - apply mask_fn
    - compute GCC = G / (R+G+B) per observation, rename to GCC_YYYYMMdd
    - stack all GCC bands and export to Drive (one single-band-per-date stacked image)
    """
    ee_roi = shapely_to_ee(roi_geom)
    start = f'{int(year)}-01-01'
    end = f'{int(year)}-12-31'
    col = ee.ImageCollection(HLS_COLLECTION).filterDate(start, end).filterBounds(ee_roi)
    col_size = col.size().getInfo()
    print(f"[{site_name} {year}] images found:", col_size)
    if col_size == 0:
        return {'status':'no_images','site':site_name,'year':year}

    # get band names to decide availability
    first = ee.Image(col.first())
    bandnames = first.bandNames().getInfo()
    print("Example bandnames:", bandnames[:30])

    # Require B2,B3,B4 to compute GCC
    required = ['B2', 'B3', 'B4']
    if not all(b in bandnames for b in required):
        print(f"  WARNING: required bands {required} not all present for {site_name} {year}. Skipping.")
        return {'status':'missing_rgb_bands','site':site_name,'year':year}

    # Prepare mask function with available bandnames
    mask_fn = make_mask_function(bandnames)

    # Preprocess each image: mask -> clip -> compute GCC -> rename band with date
    def prep(img):
        img = ee.Image(img)
        img_masked = mask_fn(img)
        img_clip = img_masked.clip(ee_roi)
        # date string YYYYMMdd
        date = ee.Date(img_clip.get('system:time_start')).format('YYYYMMdd')

        # inside GCC prep() (per-image GCC)
        red = img_clip.select('B4').toFloat().multiply(0.0001)
        green = img_clip.select('B3').toFloat().multiply(0.0001)
        blue = img_clip.select('B2').toFloat().multiply(0.0001)

        eps = ee.Image.constant(1e-6)
        denom = red.add(green).add(blue).add(eps)
        gcc = green.divide(denom).rename(ee.String('GCC_').cat(date))


        # set index and return single-band image
        return gcc.set({'system:index': ee.String(site_name).cat('_').cat(date)})

    prepared = col.map(prep)
    prepared_count = prepared.size().getInfo()
    print(f"[{site_name} {year}] prepared (GCC computed & clipped) images:", prepared_count)
    if prepared_count == 0:
        return {'status':'no_clear_images','site':site_name,'year':year}

    # Combine into single image: every observation's GCC becomes a separate band
    stacked = prepared.toBands()

    # Start export to Drive (single image with many GCC_* bands)
    export_name = f"{site_name}_{year}_GCC"
    task = ee.batch.Export.image.toDrive(
        image=stacked,
        description=export_name,
        folder=out_drive_folder,
        fileNamePrefix=export_name,
        region=ee_roi,
        scale=scale,
        maxPixels=1e13
    )
    task.start()
    print("GCC Export started:", export_name, " -> Drive folder:", out_drive_folder)
    return {'status':'export_started','site':site_name,'year':year,'task_id': getattr(task, 'id', None)}


# ---------- 按月合成的备选方案（当波段太多时） ----------
def process_site_year_monthly(site_name, year, roi_geom, use_bands, mask_fn, out_drive_folder=OUT_DRIVE_FOLDER, scale=EXPORT_SCALE):
    ee_roi = shapely_to_ee(roi_geom)
    # 获取该年影像集合
    start = f'{int(year)}-01-01'
    end = f'{int(year)}-12-31'
    col = ee.ImageCollection(HLS_COLLECTION).filterDate(start, end).filterBounds(ee_roi)

    # 为每个月生成中位数合成
    months = list(range(1,13))
    monthly_images = []
    for m in months:
        s = ee.Date.fromYMD(int(year), m, 1)
        e = s.advance(1, 'month')
        sub = col.filterDate(s, e)
        # 预处理（掩膜 -> 选波段 -> 裁剪）
        def prep_month(img):
            img = ee.Image(img)
            img_masked = mask_fn(img).clip(ee_roi)
            red = img_masked.select('B4').toFloat().multiply(0.0001)
            green = img_masked.select('B3').toFloat().multiply(0.0001)
            blue = img_masked.select('B2').toFloat().multiply(0.0001)
            eps = ee.Image.constant(1e-6)
            denom = red.add(green).add(blue).add(eps)
            return green.divide(denom)

        sub_prep = sub.map(prep_month)
        # 如果当月没有影像，跳过
        if sub_prep.size().getInfo() == 0:
            continue
        # 用中位数合成（也可改成 median/mean）
        composite = sub_prep.median()
        # 重命名波段为 B4_YYYYMM
        date_label = ee.String(str(year)).cat(ee.String('_')).cat(ee.String(str(m).zfill(2)))
        newnames = [ee.String(b).cat('_').cat(date_label) for b in use_bands]
        composite = composite.rename(newnames)
        monthly_images.append(composite)

    if len(monthly_images) == 0:
        return {'status':'no_clear_images_monthly','site':site_name,'year':year}

    # 把月合成列表合并为单影像
    stacked_monthly = ee.ImageCollection(monthly_images).toBands()

    export_name = f"{site_name}_{year}_monthly"
    task = ee.batch.Export.image.toDrive(
        image=stacked_monthly,
        description=export_name,
        folder=out_drive_folder,
        fileNamePrefix=export_name,
        region=ee_roi,
        scale=scale,
        maxPixels=1e13
    )
    task.start()
    print("Monthly-export started:", export_name)
    return {'status':'export_started_monthly','site':site_name,'year':year,'task_id': getattr(task, 'id', None)}


In [ ]:
# 批量运行（使用前面构建的 pairs 列表）
results = []
started = 0
for idx, (site, year) in enumerate(pairs, start=1):
    print(f"\n[{idx}/{len(pairs)}] 开始处理： {site} - {year}")
    if site not in roi_dict:
        print("  WARNING: site 在 shapefile 中未找到：", site)
        results.append({'site':site,'year':year,'status':'site_not_found'})
        continue
    try:
        res = process_site_year(site, year, roi_dict[site], out_drive_folder=OUT_DRIVE_FOLDER, scale=EXPORT_SCALE)
        results.append(res)
        print("  结果：", res)
        started += 1
    except Exception as e:
        print("  ERROR:", str(e))
        results.append({'site':site,'year':year,'status':'error','error_msg': str(e)})
    time.sleep(SLEEP_BETWEEN_START)

# 保存日志到 Drive
with open(LOG_PATH, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print("\n全部任务已提交（或尝试提交）。日志已保存到：", LOG_PATH)
print("本次启动的任务数量（近似）：", started)



[1/118] 开始处理： northinletsaltmarsh - 2015
[northinletsaltmarsh 2015] images found: 12
Example bandnames: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE']
[northinletsaltmarsh 2015] prepared (GCC computed & clipped) images: 12
GCC Export started: northinletsaltmarsh_2015_GCC  -> Drive folder: sential2GCC_exports_final_allData
  结果： {'status': 'export_started', 'site': 'northinletsaltmarsh', 'year': '2015', 'task_id': 'L7BCSIB46X3GSDT3HSYMZDLT'}

[2/118] 开始处理： northinletsaltmarsh - 2016
[northinletsaltmarsh 2016] images found: 72
Example bandnames: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE']
[northinletsal